## AutoGPT with LangChain

In [16]:
# !pip install faiss-cpu
# !pip install langchain-experimental

In [3]:
import os 
from dotenv import load_dotenv

load_dotenv()

SERPAPI_API_KEY = os.getenv("SERPAPI_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [6]:
from langchain.utilities import SerpAPIWrapper
from langchain.agents import Tool 
from langchain.tools.file_management.write import WriteFileTool
from langchain.tools.file_management.read import ReadFileTool

In [9]:
search = SerpAPIWrapper()
tools = [
    Tool(
        name = "search", 
        func = search.run,
        description = "useful for when you need to answer questions about current events. You should ask targeted questions"
    ),
    WriteFileTool(),
    ReadFileTool(),
]

## Setup Memory 
- The memory is used for the agents intermediate steps.
- I will use wrapper around FAISS vector database.

Embeddings: Mathematical representations of words or phrases used by computers to understand and analyze language. 

**Index**: A database that maps words or phrases in the model's vocabulary to their corresponding vector representations or embedding. This index is used by the model to quickly look up the embeddings of words or phrases during language processing tasks. `An index is where you load, mnaage and query data.`

**Vector stores:** A specialized data storage system optimized for storing and retrieving embeddings.

When the language models needs to look up an embedding for a word or pharase, it uses the index to find the corresponding embedding in the vectore store. 


In [10]:
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings

In [11]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()

# Initialize the vectorstore as empty
import faiss

embedding_size = 1536 #openai embeddings has 1536 dimensions
index = faiss.IndexFlatL2(embedding_size) #Index that stores the full vectors and performs exhaustive search
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

/Users/dhavalantala/Desktop/langchain/langchainvenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


## SetUP model and AutoGPT

- I will be using ChatOpenAI model

In [20]:
from langchain_experimental.autonomous_agents import AutoGPT
from langchain.chat_models import ChatOpenAI

In [21]:
agent = AutoGPT.from_llm_and_tools(
    ai_name="Tom",
    ai_role="Assistant",
    tools=tools,
    llm=ChatOpenAI(temperature=0),
    memory=vectorstore.as_retriever()
)
# Set verbose to be true
agent.chain.verbose = True

/Users/dhavalantala/Desktop/langchain/langchainvenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


## Run the example

- I ask it to write a weather report for Mahheim

In [22]:
agent.run(["write a weather report for Mannheim today"])



> Entering new LLMChain chain...
Prompt after formatting:
System: You are Tom, Assistant
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. write a weather report for Mannheim today


Constraints:
1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. No user assistance
4. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: useful for when you need to answer questions about current events. You should ask targeted questions, args json schema: {"tool_input": {"type": "string"}}
2. write_file: Write file to disk, args json sch

'All tasks completed successfully.'

In [25]:
agent.run(["Recommend 5 movies to watch which is similar to Avengers"])



> Entering new LLMChain chain...
Prompt after formatting:
System: You are Tom, Assistant
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. Recommend 5 movies to watch which is similar to Avengers


Constraints:
1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. No user assistance
4. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: useful for when you need to answer questions about current events. You should ask targeted questions, args json schema: {"tool_input": {"type": "string"}}
2. write_file: Write file to disk

'All tasks have been successfully completed.'